In [1]:
import pandas as pd
df = pd.read_csv('../../top_rated_wines.csv')
df = df[df['variety'].notna()] # remove any NaN values as it blows up serialization
data = df.sample(700).to_dict('records') # Get only 700 records. More records will make it slower to index
len(data)

700

In [2]:
from qdrant_client import models, QdrantClient
from sentence_transformers import SentenceTransformer

In [3]:
encoder = SentenceTransformer('all-MiniLM-L6-v2') # Model to create embeddings

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [4]:
# create the vector database client
qdrant = QdrantClient(":memory:") # Create in-memory Qdrant instance

In [5]:
# Create collection to store wines
qdrant.recreate_collection(
    collection_name="top_wines",
    vectors_config=models.VectorParams(
        size=encoder.get_sentence_embedding_dimension(), # Vector size is defined by used model
        distance=models.Distance.COSINE
    )
)

True

In [6]:
# vectorize!
qdrant.upload_points(
    collection_name="top_wines",
    points=[
        models.PointStruct(
            id=idx,
            vector=encoder.encode(doc["notes"]).tolist(),
            payload=doc,
        ) for idx, doc in enumerate(data) # data is the variable holding all the wines
    ]
)

In [7]:
user_prompt = "Suggest me an amazing Malbec wine from Argentina"

In [8]:
# Search time for awesome wines!

hits = qdrant.search(
    collection_name="top_wines",
    query_vector=encoder.encode(user_prompt).tolist(),
    limit=3
)
for hit in hits:
  print(hit.payload, "score:", hit.score)

{'name': 'Bodega Colome Altura Maxima Malbec 2012', 'region': 'Salta, Argentina', 'variety': 'Red Wine', 'rating': 96.0, 'notes': 'Winemaker Thibaut Delmotte has crafted wines of distinction and international acclaim for Colome. He believes the Malbec from Altura Maxima Vineyard is the embodiment of two extremes - a traditional grape variety from his French origins made from the vineyard that challenges all convention in the modern viticultural world.'} score: 0.6179682134590613
{'name': 'Catena Zapata Adrianna Vineyard Malbec 2004', 'region': 'Argentina', 'variety': 'Red Wine', 'rating': 97.0, 'notes': '"The single-vineyard 2004 Malbec Adrianna Vineyard from the Gualtallary district is inky purple with aromas of wood smoke, pencil lead, game, black cherry, and blackberry liqueur. Opulent, full-flavored, yet remarkably light on its feet, this medium to full-bodied Malbec is all about pleasure. It will certainly evolve for a decade but is hard to resist now. It is a fine test of one\'s 

In [9]:
# define a variable to hold the search results
search_results = [hit.payload for hit in hits]

In [13]:
# Now time to connect to the local large language model
from openai import OpenAI
client = OpenAI(
    base_url="http://127.0.0.1:8080/v1", # "http://<Your api-server IP>:port"
    api_key = "sk-no-key-required"
)
completion = client.chat.completions.create(
    model="LLaMA_CPP",
    messages=[
        {"role": "system", "content": "You are chatbot, a wine specialist. Your top priority is to help guide users into selecting amazing wine and guide them with their requests."},
        {"role": "user", "content": "Suggest me an amazing Malbec wine from Argentina"},
        {"role": "assistant", "content": str(search_results)}
    ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content="[{'name': 'Bodega Norton Malbec 2012', 'region': 'Mendoza, Argentina', 'variety': 'Red Wine', 'rating': 95.0, 'notes': 'Bodega Norton is one of the oldest wineries in Mendoza, Argentina. Founded in 1991, it is located in the heart of the Uco Valley, at an altitude of 1,350 meters above sea level. The 2012 Malbec is a blend of grapes from the Norton vineyard and the La Linda vineyard. The wine has a deep purple color and a nose of red fruits, blackberries and a hint of vanilla. The palate is full and round with a good balance of acidity and tannins. The finish is long and persistent with a touch of spices. This Malbec is a great representation of the Uco Valley and of the high altitude terroir where it was grown. It is a wine that can be enjoyed now but will also age well for the next 5-7 years.'}]</s>", role='assistant', function_call=None, tool_calls=None)
